In [1]:
import mysql.connector
import getpass
import xlsxwriter
import re
import traceback

In [2]:
def get_paramnames(table):
    #Return the parameters for the given table in the SQL set.
    query = ("SELECT `COLUMN_NAME` FROM `INFORMATION_SCHEMA`.`COLUMNS` WHERE TABLE_SCHEMA = 'certrus' AND `TABLE_NAME`= '" + table + "'")
    try:    
        cur.execute(query)    
        result = cur.fetchall();
        ret = ''
        for row in result:
            if (result.index(row) == len(result)-1):
                ret = ret + row[0]
            else:
                ret = ret + row[0] + ","
    except Exception as e:
        ret = -1
        print("##EXCEPTION CAUGHT: if you do not understand the returned code, contact your system administrator. ")
        traceback.print_exc()
    return ret;
    #if that doesn't work, see if declaring ret elsewhere helps. i dunno.

    
def insert_sql(table, parameters, col_names):
    parse_par = "("
    sql = "INSERT INTO " + table + " (" + col_names
    sql = sql + ") VALUES "
    for x in parameters:
        if(x.isdigit()):
            parse_par = parse_par + x
        else:
            parse_par = parse_par + "\"" + x + "\""
        if(parameters.index(x) == len(parameters) -1):
            parse_par = parse_par + ")"
        else:
            parse_par = parse_par + ", "
    sql = sql + parse_par
    try:
        print("SQL: " + sql)
        ret = cur.execute(sql)
        cnx.commit()
    except Exception as e:
        ret = -1
        cnx.rollback()
        print("##EXCEPTION CAUGHT: if you do not understand the returned code, contact your system administrator. ")
        traceback.print_exc()
    return ret;

    
def update_sql(table, param_check, param_change, param_newvalues):
    sql = "UPDATE " + table + " SET "
    #for x in param_change: #the fact that python is not c-base is throwing me off a little bit
    y = 0
    while(y < len(param_change)):
        sql = sql + param_change[y] + " = "
        #sql = sql + param_newvalues[y]
        if(param_newvalues[y].isdigit()):
            sql = sql + param_newvalues[y]
        else:
            sql = sql + "\"" + param_newvalues[y] + "\""
        if(y != len(param_change)-1):
            sql = sql + ", "
        y+=1
    sql = sql + " WHERE "
    for x in param_check:
        sql = sql + x
        if(param_check.index(x) != len(param_check)-1):
            sql = sql + " AND "
    try:
        print("SQL: " + sql)
        ret = cur.execute(sql)
        cnx.commit()
    except Exception as e:
        ret = -1
        cnx.rollback()
        print("##EXCEPTION CAUGHT: if you do not understand the returned code, contact your system administrator. ")
        traceback.print_exc()
    return ret;


def delete_sql(table, param_check):
    sql = "DELETE FROM " + table + " WHERE "
    for x in param_check:
        sql = sql + x
        if(param_check.index(x) != len(param_check)-1):
            sql = sql + " AND "
    try:
        print("SQL: " + sql)
        ret = cur.execute(sql)
        cnx.commit()
    except Exception as e:
        ret = -1
        cnx.rollback()
        print("##EXCEPTION CAUGHT: if you do not understand the returned code, contact your system administrator. ")
        traceback.print_exc()
    return ret;


def get_valid_filename(s):
    s = str(s).strip().replace(' ', '_')
    return re.sub(r'(?u)[^-\w.]', '', s);


def make_report(filename, param_include, col_names, param_where):
    print('Generating report... this may take awhile depending on sample space size.')
    
    sql = "SELECT DISTINCT "
    for x in param_include:
        sql = sql + x
        if(param_include.index(x) != len(param_include)-1):
            sql = sql + ", "
    sql = sql + " FROM " + col_names
    if(param_where[0] != "null"):
        sql = sql + " WHERE "
        for x in param_where:
            sql = sql + x
            if (param_where.index(x) != len(param_where)-1):
                sql = sql + " AND "
    try:
        print("SQL: " + sql)
        ret = cur.execute(sql)
        query_resp = cur.fetchall()
        print('SQL query processed, writing xlsx file...')

        workbook = xlsxwriter.Workbook(filename) 
        worksheet = workbook.add_worksheet() 

        row = 0
        col = 0        
        for item in query_resp:
            for x in item: 
                worksheet.write(row, col+item.index(x), x) 
            row += 1
        workbook.close()
    except Exception as e:
        ret = -1
        print("##EXCEPTION CAUGHT: if you do not understand the returned code, contact your system administrator. ")
        traceback.print_exc()
    return ret;
        
            


In [3]:
#functions as sort of a global variable within main. i dunno if this is required; i'm coming straight outta c-base
table = ''

#List of possible tables in the database. This shouldn't need modifications. Ever.
#The second set is tables that should probably not be added to directly.
db_tables = ('certificates', 'challenge', 'challenge_event', 'challenge_skills', 'contact_info', 'counselling', 'earns_cert', 'has_skill', 'mentor', 'prereq', 'score', 'skillmaster', 'skills', 'trainee')
db_inaccess = ('earns_cert', 'has_skill', 'mentor', 'skillmaster')

In [4]:
print('=== Welcome to the CertRUs Database Utility! ===')
is_valid = False

while True:
    try:
        uname = input(prompt = 'Please enter your username\n> ')
        pword = getpass.getpass(prompt = 'Please enter your password\n> ')
        mydb = {
         'host':'localhost',
         'user':uname,
         'passwd':pword,
         'auth_plugin':'mysql_native_password',
         'database':'certrus'
        }

        cnx = mysql.connector.connect(**mydb)
        cur = cnx.cursor()
    except Exception as error:
        print('\\\\\\Error: Username-password combination invalid!')
    else:
        is_valid = True
    if(is_valid == True):
        break;

=== Welcome to the CertRUs Database Utility! ===
Please enter your username
> root
Please enter your password
> ········


In [5]:
 while True:
    print('\n=========================================================================')
    print('---- Welcome,', uname)
    print('---- What would you like to do?')
    print('\'A\': INSERT New data entries')
    print('\'B\': UPDATE Already existing entries')
    print('\'C\': DELETE From existing entries')
    print('\'D\': Generate a report of a dataset')
    print('')
    print('\'Q\': To exit the utility')
    print('Type \'info\' for information on the accessible data tables.')

    arg = input(prompt = '> ')

    if(arg.lower() == 'a'):
        print('You chose: Option A/a')
        arg = input(prompt = 'Which table should be inserted into?\n(Press ENTER to modify last accessed table.)\n> ')
        if (arg == ''):
            if(table == ''):
                print('\\\\Error: No table accessed previously. Stop.')
                continue
        else:
            table = arg
 
        #Verify that the table exists. If not, then break.
        if (table in db_tables):
            if (table in db_inaccess):
                print('Warning: It is not advised to create additions to this table, as elements here will be added automatically.')
                arg = input(prompt = 'Continue anyways? (Y/N) \n> ')
                if (arg.lower() != 'y'):
                    continue #this is pretty non-intuitive, but this restarts the loop, not continues it.
        else:
            print('\\\\Error: No such table exists. If you believe this response to be incorrect, contact your system administrator.')
            continue

        
        #function which runs the following sql query using table as parameter:
        #SELECT `COLUMN_NAME` FROM `INFORMATION_SCHEMA`.`COLUMNS` WHERE `TABLE_NAME`='trainee';
        #return some set of the column names in order (tuple? idk.)
        col_names = get_paramnames(table);
        
        print('Parameter names of table \"' + table + '\", in order: \n' + col_names)
        arg = input(prompt = 'Enter the values for the new query, in order based on the parameters, separated by commas (,)\nNOTE: Any space characters will be included as part of the entry!\n> ')
        parameters = arg.split(',')
        
        #take the components and try to add it as an add statement to the table in question.
        result = insert_sql(table, parameters, col_names)
        
        if(result == -1): #use this to denote a failure in submitting a query
            print('\\\\Error: Could not make database changes. Make sure your syntax is correct, and that your account has proper privileges.')
        else:
            print('Insertion successful. Return value: ', result)
            

            
    elif(arg.lower() == 'b'):
        print('You chose: Option B/b.')
        arg = input(prompt = 'Which table should be updated?\n(Press ENTER to modify last accessed table.)\n> ')
        if (arg == ''):
            if(table == ''):
                print('\\\\Error: No table accessed previously. Stop.')
                continue
        else:
            table = arg
        
        #Verify that the table exists. If not, then break.
        if (table in db_tables):
            if (table in db_inaccess):
                print('Warning: It is not advised to create additions to this table, as elements here will be added automatically.')
                arg = input(prompt = 'Continue anyways? (Y/N) \n> ')
                if (arg.lower() != 'y'):
                    continue
        else:
            print('\\\\Error: No such table exists. If you believe this response to be incorrect, contact your system administrator.')
            continue
        
        #see: above for info about this.
        col_names = get_paramnames(table)
        
        print('Parameter names of table \"' + table + '\", in order: \n' + col_names)
        arg = input(prompt = 'For which parameters checked should a statement be updated? Also include conditional statements. \nEx: \"income > 50000\"\n(Delineate multiple by comma separation) (,)\n> ')
        param_check = arg.split(',')
        arg = input(prompt = 'Which parameters should be updated where last statement is true?\n(Delineate multiple by comma separation) (,)\n> ')
        param_change = arg.split(',')
        arg = input(prompt = 'What should the values of the given parameter be changed to?\n(Delineate multiple by comma separation) (,)\nNote: Make sure that each values corresponds to the order of parameters in last statement!\n> ')
        param_newvalues = arg.split(',')
        
        if(len(param_change) != len(param_newvalues)):
            print('\\\\Error: param_change != param_newvalues ! (Cannot make corresponding queries.)')
            continue
        
        #function call, define this later.
        #take the components and try to add it as an add statement to the table in question.
        result = update_sql(table, param_check, param_change, param_newvalues)
        
        if(result == -1): #use this to denote a failure in submitting a query
            print('\\\\Error: Could not make database changes. Make sure your syntax is correct, and that your account has proper privileges.')
        else:
            print('Update successful. Return value: ', result)


                        
    elif(arg.lower() == 'c'):
        print('You chose: Option C/c')
        arg = input(prompt = 'Which table should be deleted from?\n(Press ENTER to modify last accessed table.)\n> ')
        if (arg == ''):
            if(table == ''):
                print('\\\\Error: No table accessed previously. Stop.')
                continue
        else:
            table = arg
        
        #Verify that the table exists. If not, then break.
        if (table in db_tables):
            if (table in db_inaccess):
                print('Warning: It is not advised to create additions to this table, as elements here will be added automatically.')
                arg = input(prompt = 'Continue anyways? (Y/N) \n> ')
                if (arg.lower() != 'y'):
                    continue
        else:
            print('\\\\Error: No such table exists. If you believe this response to be incorrect, contact your system administrator.')
            continue
        
        #see: above for info about this.
        col_names = get_paramnames(table);
        
        print('Parameter names of table \"' + table + '\", in order: \n' + col_names)
        arg = input(prompt = 'For which parameters checked should a statement be deleted? Also include conditional statements. \nEx: \"income > 50000\"\n(Delineate multiple by comma separation) (,)\n> ')
        param_check = arg.split(',')
        
        #function call, define this later.
        #take the components and try to add it as an add statement to the table in question.
        result = delete_sql(table, param_check)
        
        if(result == -1): #use this to denote a failure in submitting a query
            print('\\\\Error: Could not make database changes. Make sure your syntax is correct, and your account has proper privileges.')
            continue
        else:
            print('Removal successful. Return value: ', result)
            
            
            
    elif(arg.lower() == 'd'):
        print('You chose: Option D/d')
        arg = input(prompt = 'What should the name of the report\'s output xlsx file be?\n> ')
        filename = get_valid_filename(arg)
        filename = filename + '.xlsx'
        
        arg = input(prompt = 'Which tables should be included into the report?\n(Delineate multiple by comma separation) (,)\nThey will be connected via natural join.\n> ')
        table_list = arg.split(',')        
        col_names = ''
        for x in table_list:
            col_names = col_names + x
            if (table_list.index(x) != len(table_list)-1):
                col_names = col_names + " NATURAL JOIN "
        
        print_col = ''
        print('Parameter names of constituent tables, in order:')
        for x in table_list:
            p_names = get_paramnames(x)
            print(x + ": " + p_names)
        
        arg = input(prompt = 'Which parameters should be included in report? (Press ENTER to include all)\n(Delineate multiple by comma separation) (,)\nFor columns appearing in both sets, make sure to specify which by dot operator (Ex: account.balance)\n> ')
        if (arg == ''):
            param_include = ['*',]
        else:
            param_include = arg.split(',')
        
        arg = input(prompt = 'Enter conditional statements for query inclusion, in parameter-value format. \nEx: \"income > 50000\"\n(Delineate multiple by comma separation) (,)\n(Press ENTER to include no special conditions)\n> ')
        if (arg == ''):
            param_where = ['null',] #Remember to still treat these as lists!
        else:
            param_where = arg.split(',')
            
        result = make_report(filename, param_include, col_names, param_where)
        
        if(result == -1): #use this to denote a failure in submitting a query
            print('\\\\Error: Could not generate report. Make sure your syntax is correct, and your account has proper privileges.')
            continue
        else:
            print('Report successfully generated. Return value: ', result)
  

            
    elif(arg.lower() == 'q'):
        print('Logging off. Goodbye, ' + uname + '.')
        arg = '' #for security purposes!
        uname = ''
        pword = ''
        break    

        
    elif(arg.lower() == 'info'):
        print('You chose: info')
        print('The list of tables is: ')
        print(*db_tables, sep = ", ")
        arg = input(prompt = 'If you want to view a table\'s list of parameters, enter its name.\nOtherwise, press ENTER to return.\n> ')
        if (arg == ''):
            continue
        table = arg
        out = get_paramnames(table)
        
        if (out == -1):
            print('\\\\Error: Could not generate report. Make sure your syntax is correct, and your account has proper privileges.')
            continue
        else:
            print('Parameters for table ' + table + ', in order: ')
            print(out)

            
    else:
        print('\\\\Error: Not a valid symbol. If you believe this response to be incorrect, contact your system administrator.')

print('Program completed. Closing out...')
cur.close()
cnx.close()


---- Welcome, root
---- What would you like to do?
'A': INSERT New data entries
'B': UPDATE Already existing entries
'C': DELETE From existing entries
'D': Generate a report of a dataset

'Q': To exit the utility
Type 'info' for information on the accessible data tables.
> A
You chose: Option A/a
Which table should be inserted into?
(Press ENTER to modify last accessed table.)
> trainee
Parameter names of table "trainee", in order: 
user_id,name
Enter the values for the new query, in order based on the parameters, separated by commas (,)
NOTE: Any space characters will be included as part of the entry!
> 666,Hajime Hinata
SQL: INSERT INTO trainee (user_id,name) VALUES (666, "Hajime Hinata")
Insertion successful. Return value:  None

---- Welcome, root
---- What would you like to do?
'A': INSERT New data entries
'B': UPDATE Already existing entries
'C': DELETE From existing entries
'D': Generate a report of a dataset

'Q': To exit the utility
Type 'info' for information on the accessib